
### Dataset

In [1]:
env = 'Breakout'
num_episodes = 25 #use them all?

In [ ]:
import anomapy.train.sssn as sssn
import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.torchutils as tu
import numpy as np

def distance(model, episode):
    z, d = sssn.distance(model, episode)
    return tu.to_numpy(z), tu.to_numpy(d)
    
def plot_latent(model, episode):
    z = tu.to_numpy(sssn.encode(model, episode))
    images = T.HWC(tu.to_numpy(episode))
    return J.scatter_image(z[:,0], z[:,1], images, scatter_colour='blue', line_colour='#b9d1fa', scale=1.5)

def order_by(model, episode):
    d, z = distance(model, episode)
    images = T.HWC(tu.to_numpy(episode))
    i = np.argsort(-d)
    return d[i], images[i]

In [ ]:
import datasets
from pprint import pprint
import numpy as np

dataset = datasets.dataset('aad.raw.{0}'.format(env))
dataset.state.transform.to_float().CHW().torch()
episodes = [x for x in dataset.state.load(num_episodes)]
for i,e in enumerate(episodes):
    print("episode:", i, e.shape)
    
print(np.sum([e.shape[0] for e in episodes]))
    
episode_test = episodes[-1]
episodes = episodes[:-1]


# Train


In [ ]:
from types import SimpleNamespace

import pyworld.toolkit.tools.wbutils as wbu
import pyworld.toolkit.tools.fileutils as fu
import pyworld.toolkit.tools.torchutils as tu
import pyworld.toolkit.tools.visutils.jupyter as J

import anomapy.train.sssn as sssn

CONFIG = sssn.default_config()
CONFIG.update(dataset.meta.to_dict())
CONFIG['latent_shape'] = 256
CONFIG['epochs'] = 12
CONFIG['batch_size'] = 128
CONFIG['total_states'] = sum([e.shape[0] for e in episodes])
CONFIG['episodes'] = len(episodes)
CONFIG = SimpleNamespace(**CONFIG)

print(CONFIG)

DRYRUN = True
PROJECT = "anomaly-detection"
RUN_ID = "{0}-{1}-{2}".format(CONFIG.model, CONFIG.name, fu.file_datetime())
RUN_TAGS = [CONFIG.model, CONFIG.name]

wbu.dryrun(DRYRUN)

optimiser = sssn.new(dryrun=DRYRUN, **CONFIG.__dict__)
model = optimiser.model

wb = wbu.WB(PROJECT, model, id=RUN_ID, tags=RUN_TAGS, config=CONFIG.__dict__)
#loss_plot = J.dynamic_plot(update_after=10)

#z = tu.to_numpy(sssn.encode(model, episodes[0]))
#plot = J.plot(z[:,0], z[:,1], mode=J.line_mode.both)

with wb:
    for i in range(CONFIG.epochs):
        print("---- epoch: ", i)
        for episode in episodes:
            for loss in sssn.epoch(optimiser, episode, CONFIG.batch_size):
                pass #loss_plot.update(None, loss['loss'])
            #z = tu.to_numpy(sssn.encode(model, episodes[0]))
            #plot.set_data(z[:,0], z[:,1])
            print("loss:", optimiser.cma())


# Load Model

In [2]:
from types import SimpleNamespace
import anomapy.train.sssn as sssn
import pyworld.toolkit.tools.wbutils as wbu


def distance(model, episode):
    z, d = sssn.distance(model, episode)
    return tu.to_numpy(z), tu.to_numpy(d)


env = "Breakout"
dryruns = sorted([r for r in wbu.dryruns() if env in r])
print(dryruns[-1])

models, config = wbu.load(dryruns[-1]) #load the most recently trained model
model = models['model.pt'].load(sssn.model(**config))
config = SimpleNamespace(**config)

Failed to initialise kaggle.
USING DEVICE: cpu
aad
aad.clean
aad.clean.BeamRider
aad.clean.BeamRider.action
aad.clean.BeamRider.state
aad.clean.Breakout
aad.clean.Breakout.action
aad.clean.Breakout.state
aad.clean.Enduro
aad.clean.Enduro.action
aad.clean.Enduro.state
aad.clean.Pong
aad.clean.Pong.action
aad.clean.Pong.state
aad.clean.Qbert
aad.clean.Qbert.action
aad.clean.Qbert.state
aad.clean.Seaquest
aad.clean.Seaquest.action
aad.clean.Seaquest.state
aad.clean.SpaceInvaders
aad.clean.SpaceInvaders.action
aad.clean.SpaceInvaders.state
aad.raw
aad.raw.BeamRider
aad.raw.BeamRider.action
aad.raw.BeamRider.state
aad.raw.Breakout
aad.raw.Breakout.action
aad.raw.Breakout.state
aad.raw.Enduro
aad.raw.Enduro.action
aad.raw.Enduro.state
aad.raw.Pong
aad.raw.Pong.action
aad.raw.Pong.state
aad.raw.Qbert
aad.raw.Qbert.action
aad.raw.Qbert.state
aad.raw.Seaquest
aad.raw.Seaquest.action
aad.raw.Seaquest.state
aad.raw.SpaceInvaders
aad.raw.SpaceInvaders.action
aad.raw.SpaceInvaders.state
aad.anomaly

# Load Anomalies


In [3]:
import datasets

dataset = datasets.dataset('aad.anomaly.{0}'.format(env))
dataset.state.transform.to_float().CHW().torch()
anoms = [(a,e[0]) for a,e in dataset.meta.anomaly.items()]
a_episodes = [d for d in dataset.state.load_files(*[e[1] for e in anoms])]
a_labels = [d for d in dataset.label.load_files(*[e[1] for e in anoms])]

loading dataset aad.anomaly.Breakout.state...


IntProgress(value=0, max=6)

loading dataset aad.anomaly.Breakout.label...


IntProgress(value=0, max=6)

In [4]:
import numpy as np
a_tlabels = [np.logical_or(l[:-1], l[1:]).astype(np.uint8) for l in a_labels] #transition labels

# ROC

In [5]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc


import plotly
import plotly.graph_objects as go

import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.visutils.plot as vplot
import pyworld.toolkit.tools.torchutils as tu
import pyworld.toolkit.tools.fileutils as fu

import numpy as n
from pprint import pprint

def roc(label, score):
    assert label.shape[0] == score.shape[0]
    fpr, tpr, _ = roc_curve(label, score)
   
    return fpr, tpr

fprs = []
tprs = []
aucs = {}
legend = []
for i in range(len(a_episodes[:-1])):
    episode = a_episodes[i]
    anomaly = anoms[i][0]
    label = a_tlabels[i]
    latent_space, score = distance(model, episode)
    
    fpr, tpr, _ = roc_curve(label, score)
    aucs[anomaly] = auc(fpr, tpr)
    fprs.append(fpr)
    tprs.append(tpr)
    legend.append(anomaly)

pprint(aucs)
legend[0] = 'flicker'
legend[1] = 'visual artefact'
legend = [l.replace('_', ' ') for l in legend]
print(legend)

plot = J.plot(fprs, tprs, legend=legend, show=False)
plot.fig.update_layout(showlegend=True, autosize=False, width=500, height=400, margin=dict(l=5,b=5,r=5,t=5))
plot.fig.update_layout(dict(legend=dict(xanchor='center', x=0.5, orientation='h')))

#path = "/home/ben/Downloads/rocs/"
#plot.fig.write_image(path + "{0}.png".format(env))
#fu.save(path + "{0}.json".format(env), aucs)



{'block': 0.9969546594805394,
 'flicker': 0.49603174603174605,
 'freeze': 0.04933858643744028,
 'freeze_skip': 0.11041058824662255,
 'split_horizontal': 0.9937982072128414}
['flicker', 'visual artefact', 'freeze', 'freeze skip', 'split horizontal']


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'flicker',
              'type': 'scattergl',
              'uid': 'b12973a5-e751-4ff6-9486-eaebc04efc0b',
              'x': array([0.        , 0.        , 0.        , 0.00226757, 0.00226757, 0.01473923,
                          0.01473923, 0.01927438, 0.01927438, 0.02040816, 0.02040816, 0.02154195,
                          0.02154195, 0.02380952, 0.02380952, 0.15306122, 0.15306122, 0.3537415 ,
                          0.3537415 , 1.        ]),
              'y': array([0.        , 0.00483092, 0.95169082, 0.95169082, 0.96135266, 0.96135266,
                          0.96618357, 0.96618357, 0.97584541, 0.97584541, 0.98067633, 0.98067633,
                          0.98550725, 0.98550725, 0.99033816, 0.99033816, 0.99516908, 0.99516908,
                          1.        , 1.        ])},
             {'mode': 'lines',
              'name': 'visual artefact',
              'type': 'scattergl',
              'uid': '062

In [ ]:
import tensorflow as tf

learning_rate = 0.01
training_steps = 100
batch_size = 256
# Logistic regression (Wx + b).

def logistic_regression(x):

    # Apply softmax to normalize the logits to a probability distribution.
    b = tf.Variable(tf.zeros([len(a_episodes[:-1])]), name="bias")
    return tf.nn.softmax(tf.matmul(latent_space, score) + b)

def cross_entropy(y_pred, y_true):

    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=len(a_episodes[:-1]))

    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# the RMSprop optimizer with the default hyper-parameters
optimizer = tf.optimizers.RMSprop(learning_rate)

In [ ]:
# Optimization process. 

def run_optimization(x, y):

# Wrap computation inside a GradientTape for automatic differentiation.

    with tf.GradientTape() as g:

        pred = logistic_regression(x)

        loss = cross_entropy(pred, y)

    # Compute gradients.

    gradients = g.gradient(loss, [latent_space, score])

  

    # Update W and b following gradients.

    optimizer.apply_gradients(zip(gradients, [latent_space, score]))

In [ ]:
# Run training for the given number of steps.

for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    # Run the optimization to update W and b values.

    run_optimization(batch_x, batch_y)

    

    if step % display_step == 0:

        pred = logistic_regression(batch_x)

        loss = cross_entropy(pred, batch_y)


        print("step: %i, loss: %f " % (step, loss))

# Histograms

In [6]:
anoms[0] = ('flicker', anoms[1])
anoms[1] = ('visual artefact', anoms[1])
anoms = [(a[0].replace('_',' '), a[1]) for a in anoms]
for a in anoms:
    print(a)

('flicker', ('flicker', 'episode(17).hdf5'))
('visual artefact', ('flicker', 'episode(17).hdf5'))
('freeze', 'episode(34).hdf5')
('freeze skip', 'episode(51).hdf5')
('split horizontal', 'episode(68).hdf5')
('split vertical', 'episode(85).hdf5')


In [7]:
import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.visutils.plot as vplot
import pyworld.toolkit.tools.torchutils as tu
import pyworld.toolkit.tools.datautils as du
import numpy as np


import plotly
import plotly.graph_objects as go


subplot = plotly.subplots.make_subplots(rows=3,cols=2, vertical_spacing = 0.08, horizontal_spacing = 0.05, subplot_titles=[a[0] for a in anoms[:-1]])



showlegend = True

def histogram(model, episode, labels, title, i, j, bins=50):
    z, d = distance(model, episode)
    d = du.normalise(d)
    
    
    d_a = d[labels==1]
    d_n = d[labels==0]
    print(d.shape, d_a.shape, d_n.shape)
    binsize = (np.max(d) - np.min(d)) / bins
    print("anomaly count: ", d_a.shape[0], "normal count: ", d_n.shape[0])
    subplot.add_trace(go.Histogram(x=d_a, marker=dict(color='red'), name='anomaly', showlegend=showlegend, xbins=dict(size=binsize)), row=i, col=j)
    subplot.add_trace(go.Histogram(x=d_n, marker=dict(color='blue'), name='normal', showlegend=showlegend, xbins=dict(size=binsize)), row=i, col=j)

histogram(model, a_episodes[0], a_tlabels[0], anoms[0], 1, 1)
showlegend=False
histogram(model, a_episodes[1], a_tlabels[1], anoms[1], 1, 2)
histogram(model, a_episodes[2], a_tlabels[2], anoms[2], 2, 1)
histogram(model, a_episodes[3], a_tlabels[3], anoms[3], 2, 2)
histogram(model, a_episodes[4], a_tlabels[4], anoms[4], 3, 1)
histogram(model, a_episodes[5], a_tlabels[5], anoms[5], 3, 2)

subplot.update_layout(yaxis_type="log")
subplot.update_layout(yaxis2_type="log")
subplot.update_layout(yaxis3_type="log")
subplot.update_layout(yaxis4_type="log")
subplot.update_layout(yaxis5_type="log")
subplot.update_layout(yaxis6_type="log")
#print(subplot)
subplot.update_layout(margin=dict(l=5,b=0,r=5,t=20))
subplot.update_layout(dict(legend=dict(xanchor='center', y = -0.03, x=0.5, orientation='h')))


(1089,) (207,) (882,)
anomaly count:  207 normal count:  882
(1402,) (252,) (1150,)
anomaly count:  252 normal count:  1150
(1647,) (600,) (1047,)
anomaly count:  600 normal count:  1047
(1445,) (396,) (1049,)
anomaly count:  396 normal count:  1049
(816,) (78,) (738,)
anomaly count:  78 normal count:  738
(646,) (61,) (585,)
anomaly count:  61 normal count:  585


# Visualise Latent Space

In [8]:
#assert config.latent_shape == 2 #otherwise... hmmm
print(config.latent_shape)

256


# Distance Graphs
Distance graphs are interactive and show the score (naturaly ordered) of each transition. 

In [9]:
import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.torchutils as tu
import numpy as np
import datasets

def plot_distance(model, episode):
    z, d = distance(model, episode)
    images = T.HWC(tu.to_numpy(episode))

    plot = J.SimplePlot(np.arange(len(d)), d)
    image1 = J.SimpleImage(images[0])
    image2 = J.SimpleImage(images[1])

    def on_hover(trace, points, state):
        i = points.point_inds[0]
        image1.set_image(images[i])
        image2.set_image(images[i+1])

    plot.on_hover(on_hover)
    d_images = J.layout_horizontal(image1.fig, image2.fig)
    d_plot = J.layout_horizontal(plot.fig)
    J.display(J.layout_vertical(d_plot, d_images))

env = "Pong"
dataset = datasets.dataset('aad.raw.{0}'.format(env))
dataset.state.transform.to_float().CHW().torch()
episode = [x for x in dataset.state.load(1)][0]
plot_distance(model, episode)

"""
for i in range(len(a_episodes)):
    a = anoms[i][0]
    print("========================== {0} ==========================".format(a))
    plot_distance(model, a_episodes[i]) 
""" 

loading dataset aad.raw.Pong.state...


IntProgress(value=0, max=1)

    'data': [{'mode': 'lines',
              'name': '1',
       …

'\nfor i in range(len(a_episodes)):\n    a = anoms[i][0]\n    print("========================== {0} ==========================".format(a))\n    plot_distance(model, a_episodes[i]) \n'

# Visualise Anomalies

In [10]:
import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.torchutils as tu
import numpy as np

for i in range(len(a_episodes)):
    a = anoms[i][0]
    print(a)
    J.images(T.HWC(tu.to_numpy(a_episodes[i])))
    #J.scatter(np.arange(len(a_labels[i])), a_labels[i])
    

flicker


interactive(children=(IntSlider(value=0, description='x', layout=Layout(width='99%'), max=1089), Output()), _d…

visual artefact


interactive(children=(IntSlider(value=0, description='x', layout=Layout(width='99%'), max=1402), Output()), _d…

freeze


...

interactive(children=(IntSlider(value=0, description='x', layout=Layout(width='99%'), max=1445), Output()), _d…

split horizontal


interactive(children=(IntSlider(value=0, description='x', layout=Layout(width='99%'), max=816), Output()), _do…

split vertical


interactive(children=(IntSlider(value=0, description='x', layout=Layout(width='99%'), max=646), Output()), _do…

#### Load and show old 2D visualisation

In [11]:
from pprint import pprint
from types import SimpleNamespace
import anomapy.train.sssn as sssn
import pyworld.toolkit.tools.wbutils as wbu
import datasets
import numpy as np

#load model
env = "Breakout"
run = "benedict-wilkins/anomapy/sssn-Breakout-20200209131829"
models, config = wbu.load(run)
config['state'] = dict(shape=config['state_shape'])
model = models['model.pt'].load(sssn.model(**config))
config = SimpleNamespace(**config)

#load data
num_episodes = 1
dataset = datasets.dataset('aad.raw.{0}'.format(env))
dataset.state.transform.to_float().CHW().torch()
episodes = [x for x in dataset.state.load(num_episodes)]



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


 -- failed to find file locally...
benedict-wilkins/anomapy/sssn-Breakout-20200209131829


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

In [12]:
#visualise
import anomapy.train.sssn as sssn
import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.torchutils as tu

a = 5
episode = a_episodes[a]
labels = a_labels[a].astype(np.uint8)
print(labels)

def plot_latent(model, episode):
    z = tu.to_numpy(sssn.encode(model, episode))
    images = T.HWC(tu.to_numpy(episode))
    return J.scatter_image(z[:,0], z[:,1], images, scatter_colour=np.array(['blue','red'])[labels], line_colour='#b9d1fa', scale=1.5)

from ipywidgets import Image, Layout, VBox, HBox, interact, IntSlider, IntProgress, HTML, Output
import ipywidgets as widgets

fig, image = plot_latent(model, episode)

fig.update_layout(width=600, height=500)
#box_layout = widgets.Layout(display='flex',flex_flow='row',align_items='center',width='100%',height='100%')
#display(HBox([fig], layout=box_layout))

#box_layout = widgets.Layout(display='flex',flex_flow='row',align_items='center',width='100%',height='100%')
#display(HBox([fig, image_widget], layout=box_layout)) #basically... this needs to be done in jupyter..?!]


[0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

FigureWidget({
    'data': [{'line': {'color': '#b9d1fa'},
              'marker': {'color': array(['blue', 'red', 'blue', ..., 'blue', 'blue', 'blue'], dtype=object)},
              'mode': 'lines+markers',
              'type': 'scattergl',
              'uid': 'b1a2841a-377a-4a0d-8ccd-89ef9c2aeb8a',
              'x': array([25.073784, 25.422682, 25.122652, ..., 25.76911 , 25.678076, 25.685606],
                         dtype=float32),
              'y': array([62.137962, 61.513325, 62.091022, ..., 60.92639 , 60.936234, 60.940464],
                         dtype=float32)}],
    'layout': {'height': 500, 'hovermode': 'closest', 'template': '...', 'width': 600}
})

In [13]:
image

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xa0\x00\x00\x00\xd2\x08\x02\x00\x00\x00BG\\\xb0\…

-----------------
-----------------
-----------------
-----------------

# OTHER DEMOS


#### dynamic plot example

In [14]:
import pyworld.toolkit.tools.visutils.jupyter as J
import time
import numpy as np

loss_plot = J.dynamic_plot(update_after=100)

for i in range(1000):
    time.sleep(0.01)
    loss_plot.update(i, np.sin(i/(np.pi*2)))
    

FigureWidget({
    'data': [{'mode': 'lines',
              'name': '1',
              'type': 'scattergl',
              'uid': '6df8aa12-33f0-48bb-92f3-29190773a23f',
              'x': [],
              'y': []}],
    'layout': {'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10},
               'plot_bgcolor': 'white',
               'template': '...',
               'xaxis': {'linecolor': 'black',
                         'linewidth': 2,
                         'mirror': True,
                         'showgrid': False,
                         'showline': True,
                         'ticks': 'outside'},
               'yaxis': {'linecolor': 'black',
                         'linewidth': 2,
                         'mirror': True,
                         'showgrid': False,
                         'showline': True,
                         'ticks': 'outside'}}
})

TypeError: object of type 'int' has no len()

In [15]:
import tensorflow as tf
z = tu.to_numpy(tf.keras.optimiser.encode(episode))
images = T.HWC(tu.to_numpy(episode))

ModuleNotFoundError: No module named 'tensorflow.python'

#### Interactive plot example

In [16]:
import pyworld.toolkit.tools.visutils.jupyter as J
import numpy as np

x = np.arange(1000)
y = np.sin(x)

plot = J.SimplePlot(x,y)
def on_hover(trace, points, state):
    ind = points.point_inds[0]
    print(ind)
    
plot.on_hover(on_hover)
plot.display()

FigureWidget({
    'data': [{'mode': 'lines',
              'name': '1',
              'type': 'scattergl',
              'uid': '1a02a171-63d9-4441-b716-14f4f9a6208d',
              'x': array([  0,   1,   2, ..., 997, 998, 999]),
              'y': array([ 0.        ,  0.84147098,  0.90929743, ..., -0.89796748, -0.85547315,
                          -0.02646075])}],
    'layout': {'hovermode': 'closest',
               'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10},
               'plot_bgcolor': 'white',
               'template': '...',
               'xaxis': {'linecolor': 'black',
                         'linewidth': 2,
                         'mirror': True,
                         'showgrid': False,
                         'showline': True,
                         'ticks': 'outside'},
               'yaxis': {'linecolor': 'black',
                         'linewidth': 2,
                         'mirror': True,
                         'showgrid': False,
                  

340
252
183
153
114
133
177
178
202
222
241
271
296


#### Interactive plot with images

In [17]:
import pyworld.toolkit.tools.visutils.jupyter as J
import numpy as np

N = 100
x = np.arange(N)
y = np.sin(x)
images = np.random.randint(0,255,size=(N, 100, 100, 3))
image = J.SimpleImage(images[0])

plot = J.SimplePlot(x,y)
def on_hover(trace, points, state):
    i = points.point_inds[0]
    image.set_image(images[i])
    
plot.on_hover(on_hover)

J.display(J.layout_horizontal(plot.fig, image.fig))



    'data': [{'mode': 'lines',
              'name': '1',
              'type': …

##### Sliding Histogram

In [18]:
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.visutils.plot as vplot
import numpy as np

N = 100
x = np.arange(N)
y = np.sin(x)
yc = np.random.uniform(0,1,size=y.shape[0])

fig = J.histogram([y,yc], show=False)
vplot.histogram_slider(fig, sizes=np.arange(0.01,0.5,0.01))



FigureWidget({
    'data': [{'name': '1',
              'type': 'histogram',
              'uid': 'd6f1d2ac-56f9-4497-a686-46c75c6c0e65',
              'x': array([ 0.        ,  0.84147098,  0.90929743,  0.14112001, -0.7568025 ,
                          -0.95892427, -0.2794155 ,  0.6569866 ,  0.98935825,  0.41211849,
                          -0.54402111, -0.99999021, -0.53657292,  0.42016704,  0.99060736,
                           0.65028784, -0.28790332, -0.96139749, -0.75098725,  0.14987721,
                           0.91294525,  0.83665564, -0.00885131, -0.8462204 , -0.90557836,
                          -0.13235175,  0.76255845,  0.95637593,  0.27090579, -0.66363388,
                          -0.98803162, -0.40403765,  0.55142668,  0.99991186,  0.52908269,
                          -0.42818267, -0.99177885, -0.64353813,  0.29636858,  0.96379539,
                           0.74511316, -0.15862267, -0.91652155, -0.83177474,  0.01770193,
                           0.85090352,  0.9

#### subplots

In [19]:
import numpy as np

N = 100
x = np.arange(N)
y = np.sin(x)
yc = np.random.uniform(0,1,size=y.shape[0])

import plotly
import plotly.graph_objects as go

def hist(x, color, name, showlegend=False):
    return go.Histogram(x=x, marker=dict(color=color), name=name, showlegend=showlegend)

c1 = 'red'
c2 = 'blue'


subplot = plotly.subplots.make_subplots(rows=3, cols=2)
trace1 = subplot.add_trace(hist(x, c1, 'anomaly', showlegend=True), row=1, col=1)
trace2 = subplot.add_trace(hist(y, c2, 'normal', showlegend=True), row=1, col=1)
trace3 = subplot.add_trace(hist(yc, c1, 'anomaly'), row=1, col=2)

subplot


### change scatter colour

In [20]:
import pyworld.toolkit.tools.visutils.jupyter as J
import numpy as np

n = 10
x = np.arange(n)
y = np.sin(x)

plot = J.SimplePlot(x,y,mode=J.line_mode.marker)
def on_hover(trace, points, state):
    ind = points.point_inds[0]
    print(ind)
    
plot.on_hover(on_hover)
#print(plot.fig)
plot.fig.data[0]['marker'] = dict(color=np.random.randint(0,3,size=n).tolist())
plot.display()



FigureWidget({
    'data': [{'marker': {'color': [1, 0, 0, 1, 1, 0, 0, 1, 1, 1]},
              'mode': 'markers',
              'name': '1',
              'type': 'scattergl',
              'uid': '1d360919-2836-492a-9e07-5350059be0e9',
              'x': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
              'y': array([ 0.        ,  0.84147098,  0.90929743,  0.14112001, -0.7568025 ,
                          -0.95892427, -0.2794155 ,  0.6569866 ,  0.98935825,  0.41211849])}],
    'layout': {'hovermode': 'closest',
               'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10},
               'plot_bgcolor': 'white',
               'template': '...',
               'xaxis': {'linecolor': 'black',
                         'linewidth': 2,
                         'mirror': True,
                         'showgrid': False,
                         'showline': True,
                         'ticks': 'outside'},
               'yaxis': {'linecolor': 'black',
                         'linewid